In [1]:
import geemap
import json
import os
import requests
import re
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd
import ee
import numpy as np
from glob import glob
import time

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#initialize GEE using your Google Account
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    
    

In [2]:
#set up GEE map canvas (will be updated with every addLayer() method)
Map = geemap.Map(center=[-27.93186,32.48897],zoom=9)
# Map.add_basemap('SATELLITE')
Map.add_basemap('CartoDB.DarkMatter')

#study area
file_path = './data/boundaries/ramsar_stlucia.geojson'
with open(file_path) as f:
    coord = json.load(f)['features'][0]['geometry']['coordinates'][0][0]
    line_geom = ee.Geometry.LineString(coord)
    
    #polygon used for clipping purposes
    poly_geom = ee.Geometry.Polygon(coord)
    
Map.addLayer(ee_object=line_geom, vis_params={'color':'red'}, name="Lesser Isimangaliso Wetland Park")
Map
    


Map(center=[-27.93186, 32.48897], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [66]:
def get_meta_s2(data_info):
    prod_id = data_info['properties']['PRODUCT_ID']
    date = pd.to_datetime(re.findall(r"(\d{8})T", prod_id)[0]).date()
    cl_cover = round(data_info['properties']['CLOUD_COVERAGE_ASSESSMENT'],2)
    tile = re.findall(r"(\d{2}[A-Z]{3})", prod_id)[0]
    df = pd.DataFrame({'id':prod_id,'date':date,'tile':tile,'cloudcover':cl_cover},index=[0])
    return df

def filter_duplicates (s2_col_list):
    s2_filtered = []
    for i in range(s2_col.size().getInfo()):
        prod_id =  s2_col_list.get(i).getInfo()['properties']['PRODUCT_ID']
        if 'OPER_PRD' not in prod_id:
            s2_filtered += [ee.Image(s2_col_list.get(i))]
    s2_filtered_col = ee.ImageCollection(s2_filtered)
    s2_filtered_list = s2_filtered_col.toList(s2_filtered_col.size())
    return s2_filtered_list

def add_wiw(img):
    wiw = img.expression(
        '((B8A/10000) <= 0.1804) && ((B12/10000) <= 0.1131)',{'B8A':img.select('B8A'),'B12':img.select('B12')}
    ).rename('RWS')
    return img.addBands(wiw)

def add_rws(img):
    mndwi = img.normalizedDifference(['B3','B11']).rename('MNDWI')
    mgrn = img.select(['B3','B4','B5']).divide(10000).reduce(ee.Reducer.min()).rename('MGRN')
    rws = img.expression(
        '(MNDWI > 0.3) && (MGRN < 0.15)',{'MNDWI':mndwi,'MGRN':mgrn}
    ).rename('RWS')
    return img.addBands(rws)

def apply_rws(img):
    mask = img.select('RWS').eq(1)
    masked_img = img.updateMask(mask)
    return masked_img

def get_labels(img):
    cluster =  img.select('cluster')
    cluster_vector = cluster.reduceToVectors(reducer = ee.Reducer.countEvery(),geometry=img.geometry(),bestEffort=True)
    labels = ee.List(cluster_vector.aggregate_array('label')).distinct().sort()
    return labels

def add_kmeans(img):

    rws_mask = img.select('RWS').eq(1)
    rgb_img = img.select(['B4', 'B3', 'B2']).updateMask(rws_mask)
    
    train_samples = rgb_img.sample(scale=10,numPixels=2000,seed=42,tileScale=4,dropNulls=True)
    k_means = ee.Clusterer.wekaKMeans(8).train(train_samples)
    img_cluster = rgb_img.cluster(k_means).rename('cluster')
    return img.addBands(img_cluster)

def plot_mask(data,layer_name):
    img = data.select(layer_name)
    img_mask = img.updateMask(img.eq(1))
    Map.addLayer(img_mask, {'palette': 'red'}, f'{layer_name}')
    
def plot_rgb(img,text):
    viz_rgb = {'bands': ['B4', 'B3', 'B2'],'gain': [0.1, 0.1, 0.1],'scale':90}
    Map.addLayer(img, viz_rgb, text+' RGB')
    
def plot_false(img,text):
    viz_rgb = {'bands': ['B6', 'B5', 'B2'],'gain': [0.1, 0.1, 0.1],'scale':90}
    Map.addLayer(img, viz_rgb, text+' False')
    
def get_dates(data):
    date= ee.Image(data).get('GENERATION_TIME')
    return date

def add_mnws(img):

    band_names = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']
    multiband_img = img.select(band_names)
    cluster_img = img.select('cluster')

    samples = multiband_img.addBands(cluster_img).stratifiedSample(numPoints=100,classBand='cluster',scale=10,tileScale=4,seed=42,dropNulls=True)

    nws_list = []
    for label in list(range(0,8)):

        scores = []

        for band in band_names:
            img_band_mu = samples.filter(ee.Filter.eq('cluster',label)).aggregate_mean(band)
            img_band_std = samples.filter(ee.Filter.eq('cluster',label)).aggregate_total_sd(band)

            img_band_raw = multiband_img.select(band)
            img_band_score = img_band_raw.subtract(img_band_mu).divide(img_band_std).pow(2).rename(f'{label} {band}')
            
            scores.append(img_band_score)

        nws = ee.Image.cat(scores).reduce('sum').divide(len(band_names)).sqrt().rename('NWS')
        nws_list.append(nws)

    mnws = ee.Image.cat(nws_list).reduce('min').rename('MNWS')
    return mnws

def get_wd(img):
    img_mnws = img.select('MNWS')
    wd_i = img.expression('MNWS < 3',{'MNWS':img_mnws})
    return wd_i

In [4]:
#get Sentinel-2 Level-1C collection for 2016

cl_pct = 100

s2_col = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(line_geom).filterDate('2017-01-01','2017-12-31') \
    .filter(ee.Filter.inList('MGRS_TILE', ['36JVP','36JVQ'])) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cl_pct))

# #filter duplicated products 
s2_col_list = s2_col.toList(s2_col.size())

print('Nr. images found: ', s2_col.size().getInfo())


Nr. images found:  210


In [67]:
unique_dates = s2_col_list.map(get_dates).distinct()

#iterave over collection and create mosaic based on similar datetime
col = []
d_days = 
for i in range(unique_dates.size().getInfo()):
    unique_date = unique_dates.get(i)
    date_day = ee.Date(unique_dates.get(i)).format('yyyy-MM-dd')
    similar_tiles = s2_col.filter(ee.Filter.eq('system:time_end', unique_date)).mosaic().clip(poly_geom).set({'DateDay':date_day})
    col.append(similar_tiles)
    
s2_mosaic_col_a = ee.ImageCollection(col)

#iterate over mosaic again and merge based on similar days
date_days = ee.List(s2_mosaic_col_a.aggregate_array('DateDay')).getInfo()
days,count = np.unique(date_days, return_counts=True)
dups = days[count>1]
non_dups = np.setdiff1d(days,dup)

dup_mos = ee.ImageCollection([s2_mosaic_col_a.filter(ee.Filter.eq('DateDay',dup)).mosaic().set({'DateDay':dup}) for dup in dups])
non_dup_mos = s2_mosaic_col_a.filter(ee.Filter.inList('DateDay', ee.List(list(non_dups))))

s2_mosaic_col_clean = non_dup_mos.merge(dup_mos)
s2_mosaic_col_list = s2_mosaic_col_clean.toList(len(days))

print('Nr. mosaics created: ', len(date_days),len(days))

In [61]:
#convert images to gif
video_args = {
  'dimensions': 480,
  'region': poly_geom,
  'framesPerSecond': 1,
  'bands': ['B11', 'B8', 'B3'],
  'gain': [0.08, 0.08, 0.08]
}

gif_name = '2017_cl100'
geemap.download_ee_video(s2_mosaic_col_clean, video_args, f'{gif_name}.gif')
geemap.add_text_to_gif(f'{gif_name}.gif', f'{gif_name}_text.gif', xy=('3%', '1%'), text_sequence=days, font_size=20, font_color='#ffffff')
os.remove(f'{gif_name}.gif')

In [ ]:
monthly_composites = []
for month in months:
    month_col = s2_mosaic_col.filter(ee.Filter.eq('Month', month)).median().set({'Month':month})
    monthly_composites.append(month_col)

In [ ]:
for i in range(len(monthly_composites)):
    img = monthly_composites[i]
    plot_rgb(img,months[i])

In [ ]:
# mnws_ts = s2_mosaic_col.map(add_rws).map(add_kmeans).map(add_mnws)
# mnws_ts_list = mnws_ts.toList(mnws_ts.size())

In [ ]:
months_10_col = s2_mosaic_col.filter(ee.Filter.eq('Month', '10'))
months_10_col_list = months_10_col.toList(months_10_col.size())

In [ ]:
img = ee.Image(months_10_col_list.get(-2))
plot_rgb(img,img.get('Date').getInfo())

In [ ]:
test = img.select(['B4','B3','B2'])
Map.addLayer(canny)

In [ ]:
img_wf = mnws_ts.map(get_wd).reduce(reducer=ee.Reducer.sum(),parallelScale=4).divide(s2_mosaic_col.size())

In [ ]:
s2_ts = s2_mosaic_col.map(add_rws).map(add_kmeans)
s2_ts_list = s2_ts.toList(s2_ts.size())

# mnws_ts_thr = []
mnws_ts_list = []
for i in range(s2_mosaic_col.size().getInfo()):
    img = ee.Image(s2_ts_list.get(i))
    mnws = add_mnws(img)
    mnws_ts_list.append(mnws.select('MNWS'))
#     mnws_ts_thr.append(get_wd(mnws))
    
# img_wf = ee.Image.cat(mnws_ts_thr).reduce('sum').divide(len(mnws_ts_thr))


In [ ]:
Map.addLayer(img_wf, {'palette':['white', '0000FF'],'min':0.1,'max':1})

In [ ]:
for i in range(len(mnws_ts_list[0:4])):
    geemap.ee_export_image(mnws_ts_list[i], filename=f'mnws_{i}.tif', scale=90, region=poly_geom, file_per_band=False)
remove_zips = list(map(lambda x:os.remove(x),glob('*.zip')))

In [ ]:
# export all bands stack to one geotiff
geemap.ee_export_image(img.select(['B4','B3','B2']), filename='rgb.tif', scale=90, region=poly_geom, file_per_band=False)
remove_zips = list(map(lambda x:os.remove(x),glob('*.zip')))

In [ ]:
# task = ee.batch.Export.image.toDrive(ee.Image(mnws_ts_list.get(0)), 
#                                      'output',
#                                      folder='GEE_output',
#                                     fileNamePrefix='mnws',
#                                     crs='EPSG:32736',
#                                     scale=10,
#                                     maxPixels=1e8,
#                                     fileFormat='GeoTIFF',
#                                     skipEmptyTiles=True,
#                                     region=ee_to_geojson(poly_geom)['coordinates'][0])
# task.start()

In [ ]:
geemap.ee_export_image_to_drive(ee.Image(mnws_ts_list.get(0)), description='result', folder='GEE_output', region=poly_geom, scale=10)